# 20201221_Statistics_Advanced

In [1]:
import os
import pandas as pd
import numpy as np
import nltk
#import nltk.tokenize
#import nltk.tag
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#from nltk.corpus import stopwords
import statistics as stat
import matplotlib.pyplot as plt

In [ ]:
#nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [1]:
def int_punct(df): 
    res_vi = []
    res_pv = []
    res_ti = []
    for k in range(len(df.axes[0])): 
        a = df['Review'][k]
        v = 0
        p = 0
        t = 0
        if pd.isna(a)==False: 
            for c in a: 
                if c==',': 
                    v = v+1
                if c==';':
                    p = p+1
                if c==' - ': # espace pour ne pas prendre en compte les mots composes 
                    t = t+1
        s = v+p+t
        if s==0: 
            res_vi.append(None)
            res_pv.append(None)
            res_ti.append(None)
        else:     
            res_vi.append(v/s*100)
            res_pv.append(p/s*100)
            res_ti.append(t/s*100)
    res = pd.DataFrame({'ID':range(len(df.axes[0])),
                        'Virgules_pct':res_vi,
                        'Point_virgules_pct':res_pv,
                        'Tirets_pct':res_ti})
    return(res)    

# Calcul des % pour la ponctuation externe aux phrases: 
def ext_punct(df): 
    res_dec = []
    res_int = []
    res_exc = []
    for k in range(len(df.axes[0])): 
        a = df['Review'][k]
        d = 0
        i = 0
        e = 0
        if pd.isna(a)==False: 
            for c in a: 
                if c=='.': 
                    d = d+1
                if c=='?':
                    i = i+1
                if c=='!': 
                    e = e+1
        s = d+i+e
        if s==0: 
            res_dec.append(None)
            res_int.append(None)
            res_exc.append(None)
        else:     
            res_dec.append(d/s*100)
            res_int.append(i/s*100)
            res_exc.append(e/s*100)
    res = pd.DataFrame({'ID':range(len(df.axes[0])),
                        'Declaratives_pct':res_dec,
                        'Interrogatives_pct':res_int,
                        'Exclamatives_pct':res_exc})
    return(res)
    
# Calcul des % des differents types de mots dans chaque critique: 
def type_mots(df):
    # Les types de mots recherches: 
    adv = ['RB','RBR','RBS']
    nn = ['NN','NNS','NNP','NNPS']
    vb = ['VB','VBD','VBG','VBN','VBP','VBZ']
    adj = ['JJ','JJR']
    ads = ['JJS']
    # Listes vides pour stocker les resultats
    res_mots = []
    res_adv = []
    res_nn = []
    res_vb = []
    res_adj = []
    res_ads = []
    for k in range(len(df.axes[0])):
    #for k in range(10):
        t = df['Review'][k]
        # Set counters to 0: 
        ca = 0
        cn = 0
        cv = 0
        cj = 0
        cs = 0
        if pd.isna(t)==False: 
            t_tok = nltk.word_tokenize(t)
            res_mots.append(len(t_tok))
            if len(t_tok)==0: # ie si pas de texte de critique
                res_adv.append(None)
                res_nn.append(None)
                res_vb.append(None)
                res_adj.append(None)
                res_ads.append(None)
            else: 
                t_tag = nltk.pos_tag(t_tok)
                for x in t_tag: 
                    if x[1] in adv: 
                        ca = ca+1
                    if x[1] in nn: 
                        cn = cn+1
                    if x[1] in vb: 
                        cv = cv+1
                    if x[1] in adj: 
                        cj = cj+1
                    if x[1] in ads: 
                        cs = cs+1
                res_adv.append(ca/len(t_tok))
                res_nn.append(cn/len(t_tok))
                res_vb.append(cv/len(t_tok))
                res_adj.append(cj/len(t_tok))
                res_ads.append(cs/len(t_tok))
        else: 
            res_mots.append(None)
            res_adv.append(None)
            res_nn.append(None)
            res_vb.append(None)
            res_adj.append(None)
            res_ads.append(None)
    res = pd.DataFrame({'ID':range(len(df.axes[0])),
                        'Nb_mots':res_mots,
                        'Adverbes':res_adv,
                        'Noms':res_nn,
                        'Verbes':res_vb,
                        'Adjectifs':res_adj,
                        'Superlatifs':res_ads})
    return res

# 15 mots les plus frequents, selon le type de mots: 
# ty = type de mot dont on souhaite obtenir le top 15
# Valeurs possibles: 'nom', 'adj', 'adv', 'sup', 'vb'
def top_15(df,ty): 
    # Def. des tags a rechercher: 
    if ty=='nom': tg = ['NN','NNS','NNP','NNPS']
    if ty=='adj': tg = ['JJ','JJR']
    if ty=='sup': tg = ['JJS']
    if ty=='adv': tg = ['RB','RBR','RBS']
    if ty=='vb': tg = ['VB','VBD','VBG','VBN','VBP','VBZ']
    l = {} # dictionnaire qui prendra en cle les mots et en valeur leur nb d'occurrences 
    for k in range(len(df.axes[0])):
        t = df['Review'][k]
        if pd.isna(t)==False: 
            t = t.lower()
            t_tok = nltk.word_tokenize(t)
            if len(t_tok)!=0:
                t_tag = nltk.pos_tag(t_tok)
                for x in t_tag: 
                    if x[1] in tg: 
                        if x[0] in l: 
                            l[x[0]] = l[x[0]]+1
                        else: 
                            l[x[0]]=1
    res = sorted(l.items(), key=lambda x: x[1]) # liste de couples
    # Retirer les 'parasites': 
    pb = ['"','“','’','—','”','t','n','‘','s']
    res = [x for x in res if x[0] not in pb]        
    res15 = res[len(res)-16:len(res)-1]
    return res15

# % verbes conjugues au passe / present / futur: 
def tps_vb(df): 
    pr = ['VBP','VBZ','VBG']
    ps = ['VBD','VBN']
    # VB: cas particulier (present / futur)
    res_pr = []
    res_ps = []
    res_fu = []
    for k in range(len(df.axes[0])):
        t = df['Review'][k]
        cpr = 0
        cps = 0
        cfu = 0
        if pd.isna(t)==False: 
            t_tok = nltk.word_tokenize(t)
            if len(t_tok)==0: # ie si pas de texte de critique
                res_pr.append(None)
                res_ps.append(None)
                res_fu.append(None)
            else: 
                t_tag = nltk.pos_tag(t_tok)
                for j in range(len(t_tag)): 
                    if t_tag[j][1] in pr: 
                        cpr = cpr+1
                    if t_tag[j][1] in ps: 
                        cps = cps+1
                    if t_tag[j][1]=='VB': 
                        if j!=0: 
                            if t_tag[j-1][0]=='will': 
                                cfu = cfu+1
                        else: 
                            cpr = cpr+1
                res_pr.append(cpr)
                res_ps.append(cps)
                res_fu.append(cfu)
        else: 
            res_pr.append(None)
            res_ps.append(None)
            res_fu.append(None)
    res = pd.DataFrame({'ID':range(len(df.axes[0])),
                        'Passe':res_ps,
                        'Present':res_pr,
                        'Futur':res_fu})
    return res

# Pronoms personnels utilises:
def pronoms_perso(df): 
    # Pronoms que l'on veut compter: 
    i = ['I','me','my','mine']
    you = ['you','yours','your']
    he = ['he','she','him','his','her','hers']
    we = ['we','us','our','ours']
    they = ['they','them','their','theirs']
    res_i = [] 
    res_you = [] 
    res_he = [] 
    res_we = [] 
    res_they = [] 
    for k in range(len(df.axes[0])):
        t = df['Review'][k]
        ci = 0
        cy = 0
        ch = 0
        cw = 0
        ct = 0
        if pd.isna(t)==False: 
            t_tok = nltk.word_tokenize(t)
            if len(t_tok)==0: # ie si pas de texte de critique
                res_i.append(None)
                res_you.append(None)
                res_he.append(None)
                res_we.append(None)
                res_they.append(None)
            else: 
                t_tag = nltk.pos_tag(t_tok)
                for x in t_tag: 
                    if x[1]=='PRP' or x[1]=='PRP$':
                        if x[0] in i: 
                            ci = ci+1
                        if x[0] in you: 
                            cy = cy+1
                        if x[0] in he: 
                            ch = ch+1
                        if x[0] in we: 
                            cw = cw+1
                        if x[0] in they: 
                            ct = ct+1
                res_i.append(ci)
                res_you.append(cy)
                res_he.append(ch)
                res_we.append(cw)
                res_they.append(ct)
        else: 
            res_i.append(None)
            res_you.append(None)
            res_he.append(None)
            res_we.append(None)
            res_they.append(None)
    res = pd.DataFrame({'ID':range(len(df.axes[0])),
                        '1e_pers_s':res_i,
                        '2e_pers':res_you,
                        '3e_pers_s':res_he,
                        '1e_pers_p':res_we,
                        '3e_pers_p':res_they})
    return res


In [7]:
C1 = pd.read_excel('20210508_corpus_I_v3.5.xlsx')
C2 = pd.read_excel('20210411_final_corpusII.xlsx')

In [3]:
C1 = pd.read_csv('../Corpus I/20201221_Corpus_I_v2.csv')
C2 = pd.read_csv('../Corpus II/20201221_Corpus_II_v1.csv')

In [8]:
C1.head(3)

,index,Newspaper,Date,Reviewer,Title,Theatre,Review,Lat,Long,Spectator_Cat,...,Word_Average_End_1,Word_Average_Start_2,Word_Average_End_2,Word_Average_Start_3,Word_Average_End_3,First_Name_Reviewer,Gender_First_Name_Reviewer,Review_Clean,Cluster_LDA,Cluster_LDA_Calculation
0,3770,SUNDAY,26/2/2012,Claudia Pritchard,THE BOMB: A PARTIAL HISTORY Two-part collectio...,T ricycle\t,"INDEPENDENT on Weeping, crying, moaning, sighi...",51.543323,-0.200005,1,...,4.56,n.a,5.1,n.a,4.7,Claudia,female,"['independent', 'weeping', 'cry', 'moaning', '...",cluster5,"(4, '0.008*""play"" + 0.007*""production"" + 0.004..."
1,12678,12.5.13 SUNDAY,NaN,Kate Bassett,NaN,NaN,﻿﻿INDEPENDENT on We used to have angels on our...,NaN,NaN,0,...,7.25,4.3,6.05,4.48,5.67,Kate,female,"['\ufeff\ufeffindependent', 'used', 'angel', '...",cluster5,"(4, '0.008*""play"" + 0.007*""production"" + 0.004..."
2,10161,9.1.11 SUNDAY,NaN,Kate Bassett,"IVONA, PRINCESS OF BURGUNDIA Revival of play",Network\t,"﻿INDEPENDENT on Veer to the right, past the in...",NaN,NaN,0,...,3.8,5.32,5.62,5.32,5.89,Kate,female,"['\ufeffindependent', 'veer', 'right', 'past',...",cluster5,"(4, '0.008*""play"" + 0.007*""production"" + 0.004..."


########################################
# 1. Syntaxe - Ponctuation interne     #
########################################
# % de virgules / point-virgules / tirets dans la critique

In [9]:
C1_PI = int_punct(C1)
C2_PI = int_punct(C2)

In [10]:
C1_PI.to_csv('20201221_C1_PI.csv',na_rep='NA')
C2_PI.to_csv('20201221_C2_PI.csv',na_rep='NA')

C1_PI.to_excel('20201221_C1_PI.xlsx',na_rep='NA')
C2_PI.to_excel('20201221_C2_PI.xlsx',na_rep='NA')

########################################
# 2. Syntaxe - Ponctuation externe     #
########################################
# % phrases declaratives / interrogatives / exclamatives dans la critique


In [11]:
C1_PE = ext_punct(C1)
C2_PE = ext_punct(C2)

In [12]:
C1_PE.to_csv('20201221_C1_PE.csv',na_rep='NA')
C2_PE.to_csv('20201221_C2_PE.csv',na_rep='NA')

C1_PE.to_excel('20201221_C1_PE.xlsx',na_rep='NA')
C2_PE.to_excel('20201221_C2_PE.xlsx',na_rep='NA')

########################################
# 3. Lexique - Types de mots           #
########################################
# % adverbes, noms, verbes, adjectifs qualificatifs, superlatifs

In [13]:
C1_type = type_mots(C1)
C2_type = type_mots(C2)

In [14]:
C1_type.to_csv('20201221_C1_typemots.csv',na_rep='NA')
C2_type.to_csv('20201221_C2_typemots.csv',na_rep='NA')
C1_type.to_excel('20201221_C1_typemots.xlsx',na_rep='NA')
C2_type.to_excel('20201221_C2_typemots.xlsx',na_rep='NA')

########################################
# 4. Lexique - Mots les plus frequents #
########################################
# 15 adverbes / noms / verbes / adjectifs / superlatifs les + frequents

In [15]:
C1_adv_top15 = top_15(C1,'adv')
# ['now','still','rather','never','then','well','here','as','only','just',
# 'even','too','also','more','so']
C1_nom_top15 =  top_15(C1,'nom')
#['London','years','theatre','audience','characters','way','story','time',
# 'performance','man','life','stage','show','production','play']
C1_vb_top15 =  top_15(C1,'vb')
#['set','see','make','being','seems','had','do','does','makes','been',
# 'was','have','be','has','are']
C1_adj_top15 =  top_15(C1,'adj')
# ['real','many','last','such','old','little','great','good','much','first',
# 'young','other','own','more','new']
C1_sup_top15 =  top_15(C1,'sup')
#['oldest','forest','youngest','earnest','strongest','funniest','worst','honest',
# 'finest','west','biggest','greatest','latest','least','most']

C2_adv_top15 = top_15(C2,'adv')
C2_nom_top15 = top_15(C2,'nom')
C2_vb_top15 = top_15(C2,'vb')
C2_adj_top15 = top_15(C2,'adj')
C2_sup_top15 = top_15(C2,'sup')

In [16]:
df_C1_adv_top15 = pd.DataFrame(C1_adv_top15, columns=['Top 15 Adv C1', 'Top 15 Adv Frequency C1'])
df_C1_nom_top15 = pd.DataFrame(C1_nom_top15, columns=['Top 15 Nom C1', 'Top 15 Nom Frequency C1'])
df_C1_vb_top15 = pd.DataFrame(C1_vb_top15, columns=['Top 15 Vb C1', 'Top 15 Vb Frequency C1'])
df_C1_adj_top15 = pd.DataFrame(C1_adj_top15, columns=['Top 15 Adj C1', 'Top 15 Adj Frequency C1'])
df_C1_sup_top15 = pd.DataFrame(C1_sup_top15, columns=['Top 15 Sup C1', 'Top 15 Sup Frequency C1'])
df_C2_adv_top15 = pd.DataFrame(C2_adv_top15, columns=['Top 15 Adv C2', 'Top 15 Adv Frequency C2'])
df_C2_nom_top15 = pd.DataFrame(C2_nom_top15, columns=['Top 15 Nom C2', 'Top 15 Nom Frequency C2'])
df_C2_vb_top15 = pd.DataFrame(C2_vb_top15, columns=['Top 15 Vb C2', 'Top 15 Vb Frequency C2'])
df_C2_adj_top15 = pd.DataFrame(C2_adj_top15, columns=['Top 15 Adj C2', 'Top 15 Adj Frequency C2'])
df_C2_sup_top15 = pd.DataFrame(C2_sup_top15, columns=['Top 15 Sup C2', 'Top 15 Sup Frequency C2'])

In [17]:
df_top15 = pd.DataFrame({'Top 15 Adv C1':df_C1_adv_top15['Top 15 Adv C1'],
                        'Top 15 Adv Frequency C1':df_C1_adv_top15['Top 15 Adv Frequency C1'],
                        'Top 15 Nom C1':df_C1_nom_top15['Top 15 Nom C1'],
                        'Top 15 Nom Frequency C1':df_C1_nom_top15['Top 15 Nom Frequency C1'],
                        'Top 15 Vb C1':df_C1_vb_top15['Top 15 Vb C1'],
                        'Top 15 Vb Frequency C1':df_C1_vb_top15['Top 15 Vb Frequency C1'],
                        'Top 15 Adj C1':df_C1_adj_top15['Top 15 Adj C1'],
                        'Top 15 Adj Frequency C1':df_C1_adj_top15['Top 15 Adj Frequency C1'],
                        'Top 15 Sup C1':df_C1_sup_top15['Top 15 Sup C1'],
                        'Top 15 Sup Frequency C1':df_C1_sup_top15['Top 15 Sup Frequency C1'],
                        'Top 15 Adv C2':df_C2_adv_top15['Top 15 Adv C2'],
                        'Top 15 Adv Frequency C2':df_C2_adv_top15['Top 15 Adv Frequency C2'],
                        'Top 15 Nom C2':df_C2_nom_top15['Top 15 Nom C2'],
                        'Top 15 Nom Frequency C2':df_C2_nom_top15['Top 15 Nom Frequency C2'],
                        'Top 15 Vb C2':df_C2_vb_top15['Top 15 Vb C2'],
                        'Top 15 Vb Frequency C2':df_C2_vb_top15['Top 15 Vb Frequency C2'],
                        'Top 15 Adj C2':df_C2_adj_top15['Top 15 Adj C2'],
                        'Top 15 Adj Frequency C2':df_C2_adj_top15['Top 15 Adj Frequency C2'],
                        'Top 15 Sup C2':df_C2_sup_top15['Top 15 Sup C2'],
                        'Top 15 Sup Frequency C2':df_C2_sup_top15['Top 15 Sup Frequency C2']
                        })

In [18]:
df_top15.head(3)

,Top 15 Adv C1,Top 15 Adv Frequency C1,Top 15 Nom C1,Top 15 Nom Frequency C1,Top 15 Vb C1,Top 15 Vb Frequency C1,Top 15 Adj C1,Top 15 Adj Frequency C1,Top 15 Sup C1,Top 15 Sup Frequency C1,Top 15 Adv C2,Top 15 Adv Frequency C2,Top 15 Nom C2,Top 15 Nom Frequency C2,Top 15 Vb C2,Top 15 Vb Frequency C2,Top 15 Adj C2,Top 15 Adj Frequency C2,Top 15 Sup C2,Top 15 Sup Frequency C2
0,very,6600,cast,7019,get,6041,last,4786,closest,121,never,8179,people,10636,make,9348,same,5873,closest,192
1,never,6718,end,7232,make,6526,many,4901,oldest,130,then,8746,cast,11877,get,9977,musical,6419,oldest,200
2,then,7320,world,7301,had,6659,musical,5442,strongest,179,n't,8795,performance,11986,does,10142,such,6476,earnest,212


In [19]:
df_top15.to_csv('20201221_top_15.csv',na_rep='NA')
df_top15.to_excel('20201221_top_15.xlsx',na_rep='NA')

########################################
# 5. Syntaxe - Temps des verbes        #
########################################
# % verbes passe / present / futur
# present = VBP, VBZ, VBG, VB sans 'will' devant
# passe = VBD, VBN
# futur = VB avec 'will' devant

In [20]:
C1_vb = tps_vb(C1)
C2_vb = tps_vb(C2)


# Stockage: 
C1_vb.to_csv('20201221_C1_vb.csv',na_rep='NA')
C2_vb.to_csv('20201221_C2_vb.csv',na_rep='NA')


C1_vb.to_excel('20201221_C1_vb.xlsx',na_rep='NA')
C2_vb.to_excel('20201221_C2_vb.xlsx',na_rep='NA')

########################################
# 6. Lexique - Emotions                #
########################################
# Quels pronoms personnels sont utilises ?
# PRP = personal pronouns ; PRP$ = possessive personal pronouns 

In [21]:
C1_pp = pronoms_perso(C1)
C2_pp = pronoms_perso(C2)

# Stockage: 
C1_pp.to_csv('20201221_C1_pp.csv',na_rep='NA')
C2_pp.to_csv('20201221_C2_pp.csv',na_rep='NA')
C1_pp.to_csv('20201221_C1_pp.xlsx',na_rep='NA')
C2_pp.to_csv('20201221_C2_pp.xlsx',na_rep='NA')